In [ ]:
# -*- coding: utf-8 -*-
import matplotlib.pyplot as plt
import numpy as np
import gzip
from inception_v3_archisen import InceptionV3  #this inputes are uint8 to check for shalpley problems
import keras
from skimage.segmentation import slic

In [ ]:
from keras.layers import Input
Input_model = Input((299,299,3))  #299 can be anynumber or even None if undefined
Incept_model = InceptionV3(include_top=False, #chop of top
                weights='imagenet', #get pretrained weights
                input_tensor=Input_model, #trying uint8 image format
                input_shape=(299,299,3), #ignore
                pooling=None, #ignore
                classes=1000) # classes: optional number only if top is true

output = Incept_model.output
output # shape of tensor is  (batch_size, Length,Width, Channels) 
output2 = keras.layers.GlobalAveragePooling2D()(output) 
output2 #global max pooling finds the max point in each channel. Frequently used, max finds the strongest signal. Average is also reasonable
output2 = keras.layers.Dropout(0.5)(output2)
output3 = keras.layers.Dense(96,activation='relu') (output2) # dense neural network with 50 layers
output4 = keras.layers.Dense(4, activation="softmax") (output3) # output 3 classes in probabiltites with softmax

from keras.models import Model
from keras.layers import Lambda
model = Model(Input_model,output4) #using logits layer
model.compile(loss = "categorical_crossentropy",
                    optimizer = 'adam',#ptimizers.SGD(lr=0.1),
                    metrics=['categorical_crossentropy',"accuracy"])

from keras.callbacks import EarlyStopping, ModelCheckpoint
model_path = 'transfer_model.h5'
callbacks = [
        EarlyStopping(
            monitor='val_categorical_crossentropy', 
            patience=3, # stop training at 3
            verbose=0),
        
        ModelCheckpoint(
            model_path , 
            monitor='val_categorical_crossentropy', 
            save_best_only=True, 
            verbose=0)
    ]
batch_size =32
from keras.callbacks import TensorBoard
tbCallBack = TensorBoard(log_dir='./log', histogram_freq=1,
                         write_graph=True,
                         write_grads=True,
                         batch_size=batch_size,
                         write_images=True)


In [ ]:
skip = 2 #increase this to reduce sample size if low memory
c0 = np.load(gzip.GzipFile('./train/c_30.npy.gz'))[::skip] # tomato late blight
c1 = np.load(gzip.GzipFile('./train/c_31.npy.gz'))[::skip] # Tomato leaf mold 
c2 = np.load(gzip.GzipFile('./train/c_32.npy.gz'))[::skip] # Septrio leaf spot
c3 = np.load(gzip.GzipFile('./train/c_37.npy.gz'))[::skip] # healthy
y0 = np.array([[1,0,0,0],]*c0.shape[0]) #one hot encoded labels
y1 = np.array([[0,1,0,0],]*c1.shape[0])
y2 = np.array([[0,0,1,0],]*c2.shape[0])
y3 = np.array([[0,0,0,1],]*c3.shape[0])

# concateneate all classes together
X ,y = np.concatenate([c0,c1,c2,c3],axis=0), np.concatenate([y0,y1,y2,y3],axis=0)
del c0,c1,c2,c3
del y0,y1,y2,y3 #clear RAM

X_tr ,y_tr = X[0::2,], y[0::2,]
X_val , y_val = X[1::2,],y[1::2,]
del X ,y
import gc
gc.collect()

In [ ]:
model.load_weights('transfer_model.h5') #trained with astar super computer for 30mins



In [ ]:
dictt= np.load('dictt_BetterSampling_100.npy').item()
print len(dictt)
print len(X_val)

In [ ]:
for i in dictt.keys():
    assert np.mean(dictt[i][0]==X_val[i])==1
print 'Pictures tally'

In [ ]:
class_names={0:'late blight',1:'leaf mold',2:'leaf spot',3:'healthy'}
from matplotlib import cm
for i in sorted(dictt.keys())[::]:
    img = dictt[i][0]
    phi = dictt[i][1]
    actual_class = class_names[np.argmax(y_val[i])]
    preds = map(lambda x : np.round(x,2),model.predict(np.array([img]))[0])
    if max(preds) > 0.9:
        if actual_class != class_names[np.argmax(preds)]:
            print 'THE FOLLOWING WAS PREDICTED WRONGLY'
        max_val = max(np.abs(np.concatenate(phi)))
        segments_slic = dictt[i][2]
        new_image = np.zeros((299,299))
        f,ax = plt.subplots(1,4,figsize=(12*2,2*4.2))
        counter  = 3
        for ii in np.argsort(preds)[::]:
          for i in range(299):
            for j in range(299):
              new_image[i,j] = phi[ii][segments_slic[i,j]]
          if class_names[ii] == actual_class:
              ax[counter].set_title('TRUE LABEL\n'+class_names[ii]+'\n'+str(preds[ii])[:4]   )
          else:
              ax[counter].set_title(class_names[ii]+'\n'+str(preds[ii])[:4]   )
          cax = ax[ii].imshow( (np.mean(img,-1) > 0.1)*1*new_image, 
                                   vmin=-max_val, vmax=max_val,cmap = plt.cm.bwr)
          counter += -1
        cb = plt.colorbar(cax,  ax=ax.ravel().tolist(), label="SHAP value", orientation="horizontal", aspect=60)
        #cbar = plt.colorbar(cax,ticks=[-1.5,-1,-0.75,-0.5,-0.25, 0,.25,.5, 75, 1,1.5], orientation='horizontal')
        plt.show()
        f,ax = plt.subplots(1,3,figsize=(12*2,2*2.5))
        ax[0].imshow(img)
        lower_bd = np.percentile(np.concatenate(new_image),20)
        upper_bd = np.percentile(np.concatenate(new_image),80)
        new_image= np.stack([new_image]*3,-1)
        ax[1].imshow(img*(new_image > upper_bd))
        ax[1].set_title('pos shap values')
        ax[2].imshow(img*(new_image < lower_bd))
        ax[2].set_title('neg shap values')
        plt.show()


In [ ]:
class_names={0:'late blight',1:'leaf mold',2:'leaf spot',3:'healthy'}
from matplotlib import cm
for i in sorted(dictt.keys())[::]:
    img = dictt[i][0]
    phi = dictt[i][1]
    actual_class = class_names[np.argmax(y_val[i])]
    preds = model.predict(np.array([img]))[0]
    if max(preds) < 0.9:
        if actual_class != class_names[np.argmax(preds)]:
            print 'THE FOLLOWING WAS PREDICTED WRONGLY'
        max_val = max(np.abs(np.concatenate(phi)))
        segments_slic = dictt[i][2]
        new_image = np.zeros((299,299))
        f,ax = plt.subplots(1,4,figsize=(12*2,2*4.2))
        counter  = 3
        for ii in np.argsort(preds)[::]:
          for i in range(299):
            for j in range(299):
              new_image[i,j] = phi[ii][segments_slic[i,j]]
          if class_names[ii] == actual_class:
              ax[counter].set_title('TRUE LABEL\n'+class_names[ii]+'\n'+str(preds[ii])[:4]   )
          else:
              ax[counter].set_title(class_names[ii]+'\n'+str(preds[ii])[:4]   )
          cax = ax[counter].imshow( (np.mean(img,-1) > 0.1)*1*new_image, 
                                   vmin=-max_val, vmax=max_val,cmap = plt.cm.bwr)
          counter += -1
        cb = plt.colorbar(cax,  ax=ax.ravel().tolist(), label="SHAP value", orientation="horizontal", aspect=60)
        #cbar = plt.colorbar(cax,ticks=[-1.5,-1,-0.75,-0.5,-0.25, 0,.25,.5, 75, 1,1.5], orientation='horizontal')
        plt.show()
        f,ax = plt.subplots(1,3,figsize=(12*2,2*2.5))
        ax[0].imshow(img)
        lower_bd = np.percentile(np.concatenate(new_image),20)
        upper_bd = np.percentile(np.concatenate(new_image),80)
        new_image= np.stack([new_image]*3,-1)
        ax[1].imshow(img*(new_image > upper_bd))
        ax[1].set_title('pos shap values')
        ax[2].imshow(img*(new_image < lower_bd))
        ax[2].set_title('neg shap values')
        plt.show()


In [ ]:
Hi Sam and Martin,

I would like to suggest myself for the tensorflow meetup talk after playing quite abit around shapley values from Scott M. Lundberg NIPS 2016 paper titled "A Unified Approach to Interpreting Model Predictions"

 :  Explainable AI : Using Shapley values